# Working with Tropospheric Emissions: Monitoring of Pollution (TEMPO)'s Image Service in folium

## Export Image

In [1]:
import requests
import folium
from datetime import datetime
import time

def convert_to_milliseconds(date_time_str):
    """Converts a date-time string in 'YYYY-MM-DD HH:MM:SS' format to milliseconds since epoch."""
    dt = datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
    milliseconds_since_epoch = int(dt.timestamp() * 1000)
    return milliseconds_since_epoch


# Function to get the JSON request of the image using the image export service
def response_export_image(image_service_url, bbox, date_time):
    
    """For more info on how to adjuste parameters, visit https://developers.arcgis.com/rest/services-reference/enterprise/export-image/"""
    
    params = {
        'bbox': ','.join(map(str, bbox)),
        'bboxSR': '4326',
        'imageSR': '4326',
        'size': '2000,1000',
        'time': date_time,
        'format': 'jpgpng',
        'pixelType': 'F64',
        'noData': '',
        'noDataInterpretation': 'esriNoDataMatchAny',
        'interpolation': 'RSP_BilinearInterpolation',
        'compression': '',
        'compressionQuality': '',
        'bandIds': '',
        'sliceId': '',
        'mosaicRule': '',
        'renderingRule': '',
        'adjustAspectRatio': 'true',
        'validateExtent': 'false',
        'lercVersion': '',
        'compressionTolerance': '',
        'f': 'json',
        'variableName': 'T2M'
    }
    response = requests.get(image_service_url, params=params)
    response.raise_for_status()
    print(response.json())
    return response.json()


image_service_url = "https://gis.earthdata.nasa.gov/image/rest/services/C2930763263-LARC_CLOUD/TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN_BETA/ImageServer/exportImage"

# User inputs defaults
date_time_str = "2024-07-10 09:16:57" #EST time
bbox = [-125.0, 22.373975499999993, -66.93457, 51.406690499999996]

# Convert time to milliseconds
date_time = convert_to_milliseconds(date_time_str) # equals 1720617417000 in ms since epoch

# retrieves images given user input
response = response_export_image(image_service_url, bbox, date_time) 

print(response)
print(bbox)
xmin , ymin, xmax, ymax = response['extent']['xmin'] ,  response['extent']['ymin'] ,  response['extent']['xmax'] ,  response['extent']['ymax']
center_lat = (ymin + ymax) / 2
center_lon = (xmin + xmax) / 2
bbox = [-125.0, 24.396308, -66.93457, 49.384358]
bounds = [[ymin, xmin], [ymax, xmax]]
print (xmin , ymin, xmax, ymax)

# Initialize the map
m = folium.Map(location=[center_lat, center_lon], zoom_start=4)

# Add new image layer

image_overlay = folium.raster_layers.ImageOverlay(
    name='TEMPO Image',
    image=response['href'],
    bounds=bounds,
    opacity=0.5,
    interactive=True,
    cross_origin=False,
    zindex=1,
)

image_overlay.add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Display the map
m.save('tempo_image_map.html')


m



{'href': 'https://gis.earthdata.nasa.gov/image/rest/directories/arcgisoutput/C2930763263-LARC_CLOUD/TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN_BETA_ImageServer/_ags_d29cb42f_62ef_4ba6_b039_d467fe0a28e7.png', 'width': 2000, 'height': 1000, 'extent': {'xmin': -125, 'ymin': 22.373975499999993, 'xmax': -66.93457, 'ymax': 51.406690499999996, 'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}, 'scale': 0}
{'href': 'https://gis.earthdata.nasa.gov/image/rest/directories/arcgisoutput/C2930763263-LARC_CLOUD/TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN_BETA_ImageServer/_ags_d29cb42f_62ef_4ba6_b039_d467fe0a28e7.png', 'width': 2000, 'height': 1000, 'extent': {'xmin': -125, 'ymin': 22.373975499999993, 'xmax': -66.93457, 'ymax': 51.406690499999996, 'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}, 'scale': 0}
[-125.0, 22.373975499999993, -66.93457, 51.406690499999996]
-125 22.373975499999993 -66.93457 51.406690499999996
